In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

In [2]:
train_df = pd.read_csv(r"C:\Users\iremo\OneDrive\Desktop\cure_the_princess_train.csv")
test_df = pd.read_csv(r"C:\Users\iremo\OneDrive\Desktop\cure_the_princess_test.csv")
val_df = pd.read_csv(r"C:\Users\iremo\OneDrive\Desktop\cure_the_princess_validation.csv")

In [3]:
X_train = train_df.drop(columns=["Cured"]).values
y_train = train_df["Cured"].values

X_test = test_df.drop(columns=["Cured"]).values
y_test = test_df["Cured"].values

X_val = val_df.drop(columns=["Cured"]).values
y_val = val_df["Cured"].values

In [4]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)

In [5]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

In [9]:
torch.manual_seed(190401069)
y_train = y_train.reshape(-1, 1)
y_val = y_val.reshape(-1, 1)

model = MLP(input_dim=X_train.shape[1], hidden_dim1=100, hidden_dim2=50, output_dim=1)

criterion = nn.BCELoss()

optimizer = optim.SGD(model.parameters(), lr=0.01)

num_epochs = 28
batch_size = 16

best_accuracy = 0.0

train_losses = []
val_losses = []
for epoch in range(num_epochs):
    train_loss = 0.0
    val_loss = 0.0
    for i in range(0, len(X_train), batch_size):
        batch_X = X_train[i:i+batch_size]
        batch_y = y_train[i:i+batch_size]

        optimizer.zero_grad()

        y_pred = model(batch_X)
        loss = criterion(y_pred, batch_y)

        loss.backward()

        optimizer.step()
        
        train_loss += loss.item()
    with torch.no_grad():
        y_val_pred = model(X_val)
        val_loss = criterion(y_val_pred, y_val)
        val_losses.append(val_loss.item())

    torch.save(model.state_dict(), 'best_model.pt')
    model.load_state_dict(torch.load('best_model.pt'))
        
    train_loss /= len(X_train)
    train_losses.append(train_loss)
    print(f"Epoch {epoch+1}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss.item():.4f}")


Epoch 1, Training Loss: 0.0378, Validation Loss: 0.4267
Epoch 2, Training Loss: 0.0275, Validation Loss: 0.3488
Epoch 3, Training Loss: 0.0225, Validation Loss: 0.3219
Epoch 4, Training Loss: 0.0195, Validation Loss: 0.2967
Epoch 5, Training Loss: 0.0174, Validation Loss: 0.2900
Epoch 6, Training Loss: 0.0159, Validation Loss: 0.2748
Epoch 7, Training Loss: 0.0145, Validation Loss: 0.2563
Epoch 8, Training Loss: 0.0134, Validation Loss: 0.2622
Epoch 9, Training Loss: 0.0125, Validation Loss: 0.2523
Epoch 10, Training Loss: 0.0117, Validation Loss: 0.2475
Epoch 11, Training Loss: 0.0110, Validation Loss: 0.2211
Epoch 12, Training Loss: 0.0103, Validation Loss: 0.2055
Epoch 13, Training Loss: 0.0097, Validation Loss: 0.2024
Epoch 14, Training Loss: 0.0092, Validation Loss: 0.1830
Epoch 15, Training Loss: 0.0087, Validation Loss: 0.1790
Epoch 16, Training Loss: 0.0083, Validation Loss: 0.1713
Epoch 17, Training Loss: 0.0080, Validation Loss: 0.1692
Epoch 18, Training Loss: 0.0077, Validat

In [13]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

model.load_state_dict(torch.load('best_model.pt'))
y_test_pred = model(X_test)
y_test_pred = (y_test_pred > 0.5).float() # binary classification için threshold
accuracy = accuracy_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)

print("Accuracy: {:.4f}".format(accuracy))
print("F1 Score: {:.4f}".format(f1))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))


Accuracy: 0.9495
F1 Score: 0.9486
Precision: 0.9704
Recall: 0.9278
